In [1]:
import os
import warnings
from langchain._api import LangChainDeprecationWarning
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
_ = load_dotenv(find_dotenv())

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

llm = ChatGroq(model="llama3-70b-8192")

### What TavilySearchResults Does (Simple & Example)
- TavilySearchResults is a tool for web search in LangChain.
- It fetches real-time search results from the web.
- It returns a list of top search results (titles, links, snippets).

In [3]:
search = TavilySearchResults(max_results=2)
# search.invoke("Who are the top stars of the 2024 Eurocup?")

### Why Does the Agent Need tools?
- Agents can use external tools to get information beyond their training data.
- tools are functions the agent can call to perform specific tasks (e.g., searching, calculations, APIs).
- Without tools, the agent relies only on the LLM's internal knowledge, which may be outdated.

In [5]:
tools = [search]
# llm_with_tools = llm.bind_tools(tools)

- Creates an agent using ReAct (Reasoning + Acting).
- The agent decides when to use a tool (like search) and how to process results.
- Uses the LLM for reasoning and generating actions.

In [6]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools,prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [12]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Run the agent

In [17]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"input": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})